In [1]:
import pandas as pd
import datetime
import numpy as np
#import gc

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Paths

MAIN_PATH = "/home/mahesh/Desktop/ML/kaggle/amex/"

# Data
FILENAME_TRAIN_DATA_CSV     = MAIN_PATH + "data/orig/train_data.csv"
FILENAME_TRAIN_LABELS_CSV   = MAIN_PATH + "data/orig/train_labels.csv"
FILENAME_CID_MAP            = MAIN_PATH + "data/processed/cid_map.csv"
FILENAME_TRAIN_DATA_FEATHER = MAIN_PATH + "data/processed/train_data.f"

In [ ]:
CDATA_SIZE = 13
NUM_C_IN_CHUNK = 1000


def process_cid(chunk, chunksize):
    for i in range(0, NUM_C_IN_CHUNK):
        cids = (chunk.iloc[(i*CDATA_SIZE):(i*CDATA_SIZE+(CDATA_SIZE))]["customer_ID"]).tolist()
        if (len(set(cids))!=1):
            print((cids))
            return (i+1)
    return 0


chunksize = CDATA_SIZE*NUM_C_IN_CHUNK
chunk_count = 0
with pd.read_csv(FILENAME_TRAIN_DATA_CSV, chunksize=chunksize) as reader:
    for chunk in reader:
        flag = process_cid(chunk, chunksize)
        if (flag != 0 ):
            print(f'chunk_size {chunk_count} failed at {flag}.')
        chunk_count += 1


In [35]:
chunksize = 10
chunk = None
with pd.read_csv(FILENAME_TRAIN_LABELS_CSV, chunksize=chunksize) as reader:
    for chunk in reader:
        chunk.head()
        #chunk.info()
        #print(chunk.dtypes)
        #chunk.describe()
        break
chunk.head()
chunk.describe()
chunk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_ID  10 non-null     object
 1   target       10 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 288.0+ bytes


In [42]:
chunksize = 10
chunk = None
with pd.read_csv(FILENAME_TRAIN_DATA_CSV, chunksize=chunksize) as reader:
    for chunk in reader:
        #chunk.head()
        #chunk.info()
        #print(chunk.dtypes)
        #chunk.describe()
        break
#chunk.describe()
chunk.info()
#chunk.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 190 entries, customer_ID to D_145
dtypes: float64(185), int64(1), object(4)
memory usage: 15.0+ KB


We need to remap the string customer-ids to our own id. We will then save this mapping in a csv file. 

TBD: We also need to count the number of occurances of each cid in the same mapping dict and then we need to write this to the CSV file.

In [18]:
def count_cids(cid_count_dict, chunk):
    for index, row in chunk.iterrows():
        cid = row['customer_ID']
        if cid in cid_count_dict:
            i = cid_count_dict[cid]
            cid_count_dict[cid] = i+1
        else:
            cid_count_dict[cid] = 1

def remap_cids(cid_map_dict, chunk):
    for index, row in chunk.iterrows():
        cid = row['customer_ID']
        if cid in cid_map_dict:
            pass
        else:
            cid_map_dict[cid] = np.uint32(remap_cids.count)
            remap_cids.count += 1
    
    
    
chunksize = 100000
cid_count_dict = {}
cid_map_dict = {}
total_rows = 0

remap_cids.count = 0

chunk_count = 0

print(datetime.datetime.now())
with pd.read_csv(FILENAME_TRAIN_DATA_CSV, chunksize=chunksize) as reader:
    for chunk in reader:
        #count_cid(cid_count_dict, chunk)
        #total_rows += len(chunk.index)
        #print(f'chunk : {chunk_count}')
        remap_cids(cid_map_dict, chunk)
        chunk_count += 1
print(datetime.datetime.now())
# Start: 2022-06-03 07:55:21.598694
# End  : 2022-06-03 08:02:06.564735

2022-07-20 11:19:11.307424
2022-07-20 11:26:59.547063


In [7]:
def write_cid_data_to_file(filename, cid_map_dict):
    with open(filename, 'w') as f:
        for key in cid_map_dict.keys():
            f.write("%s, %s\n" % (key, cid_map_dict[key]))
            
write_cid_data_to_file(FILENAME_CID_MAP, cid_map_dict)

Go though the cid->count dict and create buckets based on count of number of rows per cid. We see that each customer has between 1 to 13 rows of data.

In [8]:
cid_buckets = {}

print(datetime.datetime.now())
for cid in cid_count_dict:
    count = cid_count_dict[cid]
    if (count in cid_buckets):
        c = cid_buckets[count]
        cid_buckets[count] = c + 1
    else:
        cid_buckets[count] = 1
print(datetime.datetime.now())

total_rows = 0
total_cids = 0
for b in sorted(cid_buckets.keys()):
    print(f'{b} : {cid_buckets[b]}')
    total_rows += cid_buckets[b]*b
    total_cids += cid_buckets[b]
    
print(f'total rows:{total_rows}')
print(f'total cids:{total_cids}')


2022-06-05 18:34:54.170492
2022-06-05 18:34:54.170611
total rows:0
total cids:0


Number of rows per cid below
1 : 5120, 2 : 6098, 3 : 5778, 4 : 4673, 5 : 4671, 6 : 5515, 7 : 5198, 8 : 6110, 9 : 6411
10 : 6721, 11 : 5961, 12 : 10623, 13 : 386034

total rows:5531451

total cids:458913


If we add zeroed out rows to have 13 rows per cid, then we will add (458913 \* 13 - 5531415 = 434418 rows) which is about 7% additional rows that are all zeroed out.

Iterate over the rows and collect some stats on the columns (max, min value), in the case of enums we'll collect the data in a set.

In [29]:
#chunk.head()
#chunk.info()
#print(chunk.dtypes)
#chunk.describe()
#print(type(chunk["customer_ID"][0]))
#print((chunk["customer_ID"]))
#print((chunk["S_2"][0]))


id_time_cols = set(['customer_ID', 'S_2'])
cat_cols = set(['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'])


def process_cat_col(chunk, col_stats, col):
    vals = chunk[col].astype("category").cat.categories.values.tolist()
    n_null_count = chunk[col].isnull().sum()
    if col in col_stats:
        (c_cat_vals, c_null_count) = col_stats[col]
        col_stats[col] = (c_cat_vals.union(set(vals)), c_null_count + n_null_count)
    else:
        col_stats[col] = (set(vals), n_null_count)
    
def process_real_col( chunk, col_stats, col):
    n_min = chunk[col].min()
    n_max = chunk[col].max()
    n_nan_count = chunk[col].isnull().sum()
    if col in col_stats:
        ((c_min, c_max), c_nan_count) = col_stats[col]
        if (c_min < n_min):
            n_min = c_min
        if (c_max > n_max):
            n_max = c_max
        col_stats[col] = ((n_min, n_max), c_nan_count + n_nan_count)
    else:
        col_stats[col] = ((n_min, n_max), n_nan_count)

def process_chunk(chunk, col_stats):
    columns = list(chunk)
    for col in columns:
        if col not in id_time_cols:
            if col in cat_cols:
                process_cat_col(chunk, col_stats, col)
            else:
                process_real_col(chunk, col_stats, col)
            
chunksize = 100000
col_stats = {}

print(datetime.datetime.now())
with pd.read_csv(FILENAME_TRAIN_DATA_CSV, chunksize=chunksize) as reader:
    for chunk in reader:
        process_chunk(chunk, col_stats)
print(datetime.datetime.now())
# 2022-06-04 12:56:04.261847
# 2022-06-04 12:57:42.292620

2022-07-20 11:39:58.778921
2022-07-20 11:41:46.622170


In [10]:
TOTAL_ROWS = 5531451

for col in col_stats:
    if col in cat_cols:
        print (f'{col} values are {col_stats[col][0]}, null values are {(col_stats[col][1]/TOTAL_ROWS)*100}')
    else:
        print (f'{col} min, max is {col_stats[col][0]}, null values are {(col_stats[col][1]/TOTAL_ROWS)*100}')

P_2 min, max is (-0.4589548331054999, 1.009999947398878), null values are 0.8313370216964772
D_39 min, max is (5.026190099677308e-09, 5.389619396242631), null values are 0.0
B_1 min, max is (-7.588799098308654, 1.3240604020685645), null values are 0.0
B_2 min, max is (9.192279870617882e-09, 1.009999999641531), null values are 0.0364461332117016
R_1 min, max is (1.5342230152981531e-09, 3.256284433154198), null values are 0.0
S_3 min, max is (-0.6271320453080809, 5.482888258321006), null values are 18.44984254583472
D_41 min, max is (5.566545069335403e-10, 8.988807033179892), null values are 0.0364461332117016
B_3 min, max is (6.2852931681511845e-09, 1.6252622436744049), null values are 0.0364461332117016
D_42 min, max is (-0.0004543302584009, 4.191118756428802), null values are 85.69427804747795
D_43 min, max is (1.1545498247667041e-07, 10.111619400622503), null values are 29.981211078250535
D_44 min, max is (5.153088222797917e-10, 5.634723661789413), null values are 4.9592593335817305


In [11]:
for col in col_stats:
    if col in cat_cols:
        print (f'{col} values are {col_stats[col][0]}, null values are {(col_stats[col][1]/TOTAL_ROWS)*100}')
        for e in col_stats[col][0]:
            print(type(e))

D_63 values are {'CO', 'XZ', 'CL', 'XM', 'XL', 'CR'}, null values are 0.0
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
D_64 values are {'R', '-1', 'U', 'O'}, null values are 3.9310119532831442
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
D_66 values are {0.0, 1.0}, null values are 88.73073267755603
<class 'float'>
<class 'float'>
D_68 values are {0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0}, null values are 3.9140362989747177
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
B_30 values are {0.0, 1.0, 2.0}, null values are 0.0364461332117016
<class 'float'>
<class 'float'>
<class 'float'>
B_38 values are {1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0}, null values are 0.0364461332117016
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
D_114 values are {0.0, 1.0}, null values are 3.1947494427773107
<class 'float'>
<class 'float'>
D_116 

In [47]:
#
# Converter dictionary to be used when reading the CSV into a dataframe.
#
converter_dict = {
        "customer_ID" : (lambda x: (np.uint32(cid_map_dict[x]))),
        #"customer_ID" : (lambda x: ""),
        "S_2": pd.to_datetime
    }

dtype_dict = {}

for col_name in col_stats:
    if col_name in cat_cols:
        dtype_dict[col_name] = 'category'
    else:
        dtype_dict[col_name] = 'float32'
    

In [21]:
chunksize = 100
with pd.read_csv(FILENAME_TRAIN_DATA_CSV, chunksize=chunksize) as reader:
    for chunk in reader:
        chunk.info()
        break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 190 entries, customer_ID to D_145
dtypes: float64(185), int64(1), object(4)
memory usage: 145.0+ MB


In [28]:
chunksize = 100
with pd.read_csv(FILENAME_TRAIN_DATA_CSV, chunksize=chunksize, dtype = dtype_dict, converters = converter_dict) as reader:
    for chunk in reader:
        chunk.info()
        break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 190 entries, customer_ID to D_145
dtypes: category(11), datetime64[ns](1), float32(177), int64(1)
memory usage: 73.0 KB


<ipython-input-28-35758c9a67e2>:3: ParserWarning: Both a converter and dtype were specified for column customer_ID - only the converter will be used
  for chunk in reader:


In [18]:
print(datetime.datetime.now())
df1 =  pd.read_csv(FILENAME_TRAIN_DATA_CSV)
print(datetime.datetime.now())
df1.info(memory_usage="deep")

2022-06-05 18:42:01.412960
2022-06-05 18:43:43.532927
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 190 entries, customer_ID to D_145
dtypes: float64(185), int64(1), object(4)
memory usage: 9.2 GB


In [100]:
del df1
gc.collect()

0

In [48]:
print(datetime.datetime.now())
train_df =  pd.read_csv(FILENAME_TRAIN_DATA_CSV,  dtype = dtype_dict, converters = converter_dict)
print(datetime.datetime.now())
train_df.info(memory_usage="deep")

2022-07-22 11:38:23.914909
2022-07-22 11:44:50.024458
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 190 entries, customer_ID to D_145
dtypes: category(11), datetime64[ns](1), float32(177), int64(1)
memory usage: 3.8 GB


In [43]:
print(datetime.datetime.now())
train_df[["customer_ID"]] = train_df[["customer_ID"]].astype(np.uint32)
print(datetime.datetime.now())
train_df.info(memory_usage="deep")

2022-07-20 18:35:43.923514
2022-07-20 18:35:43.944004
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 190 entries, customer_ID to D_145
dtypes: category(11), datetime64[ns](1), float32(177), uint32(1)
memory usage: 3.8 GB


In [103]:
train_df.describe()

,customer_ID,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
count,5.531451e+06,5.485466e+06,5.531451e+06,5.531451e+06,5.529435e+06,5.531451e+06,4.510907e+06,5.529435e+06,5.529435e+06,791314.000000,...,1.946990e+05,1.946990e+05,1.946990e+05,5.429903e+06,5.490819e+06,5.429903e+06,944408.000000,5.429903e+06,5.490724e+06,5.429903e+06
mean,2.294194e+05,6.563343e-01,1.531172e-01,1.240101e-01,6.214890e-01,7.880273e-02,2.258454e-01,5.978473e-02,1.325390e-01,0.184974,...,2.427725e-01,1.424409e-02,1.646184e-01,1.789304e-01,2.664348e-02,1.645212e-01,0.390799,1.788023e-01,5.238953e-02,6.233494e-02
std,1.324902e+05,2.446494e-01,2.700709e-01,2.119869e-01,4.014876e-01,2.263971e-01,1.933475e-01,2.025443e-01,2.349929e-01,0.228185,...,2.101320e-01,9.571116e-02,2.670208e-01,3.790614e-01,1.455480e-01,3.482771e-01,0.236182,3.789498e-01,1.825135e-01,1.934937e-01
min,0.000000e+00,-4.589548e-01,5.026190e-09,-7.588799e+00,9.192280e-09,1.534223e-09,-6.271321e-01,5.566545e-10,6.285293e-09,-0.000454,...,6.316773e-08,1.078787e-08,3.307923e-08,3.767347e-10,3.725073e-09,1.650100e-10,-0.014539,5.549692e-09,2.500991e-09,1.226024e-09
25%,1.146690e+05,4.803307e-01,4.528464e-03,8.863645e-03,1.053313e-01,2.895934e-03,1.272588e-01,2.873244e-03,5.227570e-03,0.037516,...,9.314305e-03,2.532470e-03,3.517452e-03,3.027212e-03,2.555848e-03,3.026087e-03,0.199399,3.028116e-03,2.752895e-03,3.028347e-03
50%,2.293800e+05,6.942950e-01,9.056902e-03,3.132968e-02,8.143328e-01,5.782230e-03,1.639082e-01,5.746725e-03,9.777229e-03,0.120519,...,2.539468e-01,5.069830e-03,7.037814e-03,6.053010e-03,5.110523e-03,6.051590e-03,0.382136,6.053151e-03,5.508129e-03,6.053247e-03
75%,3.442090e+05,8.648159e-01,2.366407e-01,1.259019e-01,1.002403e+00,8.660590e-03,2.581017e-01,8.615665e-03,1.550507e-01,0.250869,...,2.582450e-01,7.573434e-03,5.015469e-01,9.080455e-03,7.663697e-03,9.078914e-03,0.559307,9.076287e-03,8.260448e-03,9.080930e-03
max,4.589120e+05,1.010000e+00,5.389619e+00,1.324060e+00,1.010000e+00,3.256284e+00,5.482888e+00,8.988807e+00,1.625262e+00,4.191119,...,1.759910e+00,1.009998e+00,3.005383e+00,1.010000e+00,1.010000e+00,1.339910e+00,2.229368,1.010000e+00,1.343331e+00,4.827630e+00


In [26]:
t_count = {}
for t in train_df.dtypes.tolist():
    if t in t_count:
        t_count[t] += 1
    else:
        t_count[t] = 1
        
        
print(t_count)

{dtype('uint32'): 1, dtype('<M8[ns]'): 1, dtype('float32'): 177, CategoricalDtype(categories=['CL', 'CO', 'CR', 'XL', 'XM', 'XZ'], ordered=False): 1, CategoricalDtype(categories=['-1', 'O', 'R', 'U'], ordered=False): 1, CategoricalDtype(categories=['0.0', '1.0'], ordered=False): 4, CategoricalDtype(categories=['0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0'], ordered=False): 1, CategoricalDtype(categories=['0.0', '1.0', '2.0'], ordered=False): 1, CategoricalDtype(categories=['1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0'], ordered=False): 1, CategoricalDtype(categories=['-1.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0'], ordered=False): 1, CategoricalDtype(categories=['-1.0', '0.0', '1.0'], ordered=False): 1}


Why we are using feather: https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d

In [30]:
train_df.to_feather(FILENAME_TRAIN_DATA_FEATHER)

In [32]:
train_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 190 entries, customer_ID to D_145
dtypes: category(11), datetime64[ns](1), float32(177), uint32(1)
memory usage: 3.8 GB


In [45]:
print(datetime.datetime.now())
train_labels_df =  pd.read_csv(FILENAME_TRAIN_LABELS_CSV, converters = {"customer_ID" : (lambda x: (np.uint32(cid_map_dict[x])))})
print(datetime.datetime.now())
train_labels_df.info(memory_usage="deep")

2022-07-22 11:35:44.609629
2022-07-22 11:35:45.012990
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458913 entries, 0 to 458912
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   customer_ID  458913 non-null  int64
 1   target       458913 non-null  int64
dtypes: int64(2)
memory usage: 7.0 MB


In [39]:
(train_labels_df['target']==1).sum()

118828

NameError: name 'train_df' is not defined